## Dogs vs. Cats



![](https://storage.googleapis.com/kaggle-competitions/kaggle/3362/media/woof_meow.jpg)




##  Dataset 
````
./dataset/
----> training_set/
--------> dog/
               image1
               image2
               .
               .
--------> cat/
               image1
               image2
               .
               .
----> test_set/
--------> dog/
               image1
               image2
               .
               .
--------> cat/
               image1
               image2
               .
           .
````



In [8]:
# example of loading the cifar10 dataset

from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [10]:
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = './dataset/training_set/'
validation_data_dir = './dataset/test_set/'
nb_train_samples = 8000
nb_validation_samples = 2000
epochs = 50
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [11]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [13]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [15]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save_weights('first_try.h5')

Epoch 1/50
500/500 [==============================] - 238s 475ms/step - loss: 0.6341 - accuracy: 0.6471 - val_loss: 0.5375 - val_accuracy: 0.7360
Epoch 2/50
500/500 [==============================] - 229s 459ms/step - loss: 0.5569 - accuracy: 0.7232 - val_loss: 0.5222 - val_accuracy: 0.7505
Epoch 3/50
500/500 [==============================] - 248s 495ms/step - loss: 0.5174 - accuracy: 0.7511 - val_loss: 0.5128 - val_accuracy: 0.7500
Epoch 4/50
500/500 [==============================] - 267s 534ms/step - loss: 0.4762 - accuracy: 0.7826 - val_loss: 0.4817 - val_accuracy: 0.7805
Epoch 5/50
500/500 [==============================] - 337s 673ms/step - loss: 0.4543 - accuracy: 0.7940 - val_loss: 0.5828 - val_accuracy: 0.7245
Epoch 6/50
500/500 [==============================] - 341s 683ms/step - loss: 0.4109 - accuracy: 0.8232 - val_loss: 0.5207 - val_accuracy: 0.7930
Epoch 8/50
500/500 [==============================] - 344s 689ms/step - loss: 0.4031 - accuracy: 0.8315 - val_loss: 0.5342 -

**Save your model**

In [16]:
model.save("cat_dog_blog.h5")

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
activation (Activation)      (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 64)        1

In [20]:
scores2l = model.evaluate(validation_generator, verbose=0)
print("Accuracy: %.2f%%" % (scores2l[1]*100))

Accuracy: 81.80%


**Evaluate the model**

In [62]:
model.evaluate_generator(generator=validation_generator,
steps=nb_validation_samples // batch_size)

[0.6377977952361107, 0.818]

In [68]:
STEP_SIZE_TEST=validation_generator.n//validation_generator.batch_size
validation_generator.reset()
pred=model.predict_generatord(validation_generator,
steps=STEP_SIZE_TEST,
verbose=1)

125/125 [==============================] - 13s 105ms/step


In [69]:
predicted_class_indices=np.argmax(pred,axis=1)

In [70]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [71]:
import pandas as pd
filenames=validation_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
results.to_csv("results_1.csv",index=False)

**Single prediction**

In [72]:
test_image = image.load_img('./dataset/single_prediction/12482.jpg', target_size = (150, 150))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
# train_generator.class_indices
if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'
print(prediction)

cat
